In [3]:
import json
with open('transfer_results.json', 'r') as fp:
    tx_data = json.load(fp)

In [4]:
import pandas as pd
df = pd.DataFrame(tx_data)
df['mint'] = df['from'] == '0x0000000000000000000000000000000000000000'
df.columns

Index(['from', 'to', 'value', 'transferIndex', 'borrowCap', 'supplyCap',
       'totalVariableDebt', 'variableBorrowRate', 'blockNumber',
       'blockTimestamp', 'transactionHash', 'mint'],
      dtype='object')

In [5]:
minters = df[df['mint']]['to']
len(minters)

7646

## First order flows

We avoid double counting (otherwise arb bots contracts skew the results)

### Hoarders

In [68]:
senders = set(df['from'])
hoarders = [user for user in minters if user not in senders]
len(set(hoarders))

141

### Contracts

In [7]:
minter_txs = df[df['from'].isin(minters)]
minter_txs = minter_txs.drop_duplicates(['from', 'to'])

In [8]:
minter_txs['to'].value_counts()[:20]

0x00907f9921424583e7ffbfedf84f92b7b2be4977    1142
0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d     627
0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d     449
0x9008d19f58aabd9ed0d60971565aa8510560ab41     298
0xdef171fe48cf0115b1d80b88dc8eab59176fee57     227
0x383e7acd889bf57b0d79a584009cb570534ab518     223
0xe37e799d5077682fa0a244d46e5649f71457bd09     166
0x54eebc36527fe2e5624051e3c895810d7b68bcfc     144
0xba12222222228d8ba445958a75a0704d566bf2c8     122
0x74de5d4fcbf63e00296fd95d33236b9794016631     100
0xad3b67bca8935cb510c8d18bd45f0b94f54a968f      82
0x8d8404f8cca4c8834ca3cab1e54887ae47724bee      76
0x5756880b6a1eaba0175227bf02a7e87c1e02b28c      58
0x22f9dcf4647084d6c31b2765f6910cd85c178c18      53
0x464c71f6c2f760dda6093dcb91c24c39e5d6e18c      52
0xbbec426df87537ad0bec0e899007be441427d0f8      49
0xf0d4c12a5768d806021f80a262b4d39d26c58b8d      45
0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0      43
0x92f3f71cef740ed5784874b8c70ff87ecdf33588      42
0xf081470f5c6fbccf48cc4e5b82dd9

In [9]:
import os
import requests

API_KEY = os.environ.get('ETHERSCAN_TOKEN')
contract_names = {}

In [10]:
for contract in minter_txs['to'].value_counts()[:30].index:
    URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract}&apikey={API_KEY}"
    r = requests.get(URL)
    if r.json()['result'][0]['Implementation']:
        impl_contract = r.json()['result'][0]['Implementation']
        URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={impl_contract}&apikey={API_KEY}"
        r = requests.get(URL)        
    contract_names[contract] = (r.json()['result'][0]['ContractName'])
    print(contract, (r.json()['result'][0]['ContractName']))

0x00907f9921424583e7ffbfedf84f92b7b2be4977 GhoAToken
0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d UniswapV3Pool
0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d StakeToken
0x9008d19f58aabd9ed0d60971565aa8510560ab41 GPv2Settlement
0xdef171fe48cf0115b1d80b88dc8eab59176fee57 DirectSwap
0x383e7acd889bf57b0d79a584009cb570534ab518 UniswapV3Pool
0xe37e799d5077682fa0a244d46e5649f71457bd09 
0x54eebc36527fe2e5624051e3c895810d7b68bcfc UniswapV3Pool
0xba12222222228d8ba445958a75a0704d566bf2c8 Vault
0x74de5d4fcbf63e00296fd95d33236b9794016631 Spender
0xad3b67bca8935cb510c8d18bd45f0b94f54a968f 
0x8d8404f8cca4c8834ca3cab1e54887ae47724bee 
0x5756880b6a1eaba0175227bf02a7e87c1e02b28c UpgradeableLockReleaseTokenPool
0x22f9dcf4647084d6c31b2765f6910cd85c178c18 FillQuoteTransformer
0x464c71f6c2f760dda6093dcb91c24c39e5d6e18c Collector
0xbbec426df87537ad0bec0e899007be441427d0f8 UniswapV3Pool
0xf0d4c12a5768d806021f80a262b4d39d26c58b8d CurveRouter v1.0
0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0 
0x92f3f71cef740ed5784874b

In [11]:
contract_names['0xe37e799d5077682fa0a244d46e5649f71457bd09'] = '1inch'
contract_names['0xad3b67bca8935cb510c8d18bd45f0b94f54a968f'] = '1inch'
contract_names['0x8d8404f8cca4c8834ca3cab1e54887ae47724bee'] = '1inch'
contract_names['0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0'] = '1inch'
contract_names['0x92f3f71cef740ed5784874b8c70ff87ecdf33588'] = '1inch'
contract_names['0xf081470f5c6fbccf48cc4e5b82dd926409dcdd67'] = 'Kyber Network'
contract_names['0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d'] = 'GHO Staking'
contract_names['0x9008d19f58aabd9ed0d60971565aa8510560ab41'] = 'CowSwap'
contract_names['0xdef171fe48cf0115b1d80b88dc8eab59176fee57'] = 'ParaSwap'
contract_names['0xba12222222228d8ba445958a75a0704d566bf2c8'] = 'Balancer'
contract_names['0x74de5d4fcbf63e00296fd95d33236b9794016631'] = 'AirSwap'
contract_names['0x5756880b6a1eaba0175227bf02a7e87c1e02b28c'] = 'GHO CCIP'
contract_names['0x22f9dcf4647084d6c31b2765f6910cd85c178c18'] = '0x Exchange'
contract_names['0x464c71f6c2f760dda6093dcb91c24c39e5d6e18c'] = 'Aave Collector V2'
contract_names['0xf0d4c12a5768d806021f80a262b4d39d26c58b8d'] = 'Curve'
contract_names['0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'] = "Li.Fi"
contract_names['0x14cf6d2fe3e1b326114b07d22a6f6bb59e346c67'] = "Maverick"
contract_names['0x16c6521dff6bab339122a0fe25a9116693265353'] = "Curve"
contract_names['0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61'] = "Curve"
contract_names['0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61'] = "Curve"
contract_names['0xf4d34d7fdd2ed5d57c6f583237bb038fc3839d17'] = '1inch'
contract_names['0x28104d4f703ee5b5011cefe106f54efd56f33f95'] = 'Odos'
contract_names['0x5f0000d4780a00d2dce0a00004000800cb0e5041'] = 'ParaSwap'
contract_names['0xbb33ed8c1148e62411a97d25390fd48698fd937e'] = '1inch'
contract_names

{'0x00907f9921424583e7ffbfedf84f92b7b2be4977': 'GhoAToken',
 '0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d': 'UniswapV3Pool',
 '0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d': 'GHO Staking',
 '0x9008d19f58aabd9ed0d60971565aa8510560ab41': 'CowSwap',
 '0xdef171fe48cf0115b1d80b88dc8eab59176fee57': 'ParaSwap',
 '0x383e7acd889bf57b0d79a584009cb570534ab518': 'UniswapV3Pool',
 '0xe37e799d5077682fa0a244d46e5649f71457bd09': '1inch',
 '0x54eebc36527fe2e5624051e3c895810d7b68bcfc': 'UniswapV3Pool',
 '0xba12222222228d8ba445958a75a0704d566bf2c8': 'Balancer',
 '0x74de5d4fcbf63e00296fd95d33236b9794016631': 'AirSwap',
 '0xad3b67bca8935cb510c8d18bd45f0b94f54a968f': '1inch',
 '0x8d8404f8cca4c8834ca3cab1e54887ae47724bee': '1inch',
 '0x5756880b6a1eaba0175227bf02a7e87c1e02b28c': 'GHO CCIP',
 '0x22f9dcf4647084d6c31b2765f6910cd85c178c18': '0x Exchange',
 '0x464c71f6c2f760dda6093dcb91c24c39e5d6e18c': 'Aave Collector V2',
 '0xbbec426df87537ad0bec0e899007be441427d0f8': 'UniswapV3Pool',
 '0xf0d4c12a5768d806021f80a262b

In [12]:
for contract in minter_txs['to'].value_counts()[30:60].index:
    URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract}&apikey={API_KEY}"
    r = requests.get(URL)
    if r.json()['result'][0]['Implementation']:
        impl_contract = r.json()['result'][0]['Implementation']
        URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={impl_contract}&apikey={API_KEY}"
        r = requests.get(URL)        
    #contract_names[contract] = (r.json()['result'][0]['ContractName'])
    print(contract, (r.json()['result'][0]['ContractName']))

0x6467e807db1e71b9ef04e0e3afb962e4b0900b2b FeeReceiver
0xd8533305aa8d9eb5681f15f1461e02c1601eed72 UniswapV3Pool
0x5f515f6c524b18ca30f7783fb58dd4be2e9904ec 
0x050ebe3dbb4b3a3526735b04cc3d96c80609ee7e 
0xd5e74de4385ef0eb1bb4db05a6a504f04d92e79d 
0x86152df0a0e321afb3b0b9c4deb813184f365ada Vyper_contract


KeyboardInterrupt: 

In [71]:
contract_names |= {
    #'0x6467e807db1e71b9ef04e0e3afb962e4b0900b2b': 'DefiSaver',
    '0xd8533305aa8d9eb5681f15f1461e02c1601eed72': 'UniswapV3Pool',
    "0x5f515f6c524b18ca30f7783fb58dd4be2e9904ec": "1inch",
    "0x050ebe3dbb4b3a3526735b04cc3d96c80609ee7e": "Maverick",
    "0x86152df0a0e321afb3b0b9c4deb813184f365ada": "Curve",
    "0xbf683b7aaf9accea6bff8e926e03f6ceb98b525a": "1inch",
    "0xd1742b3c4fbb096990c8950fa635aec75b30781a": "1inch",
    "0x74345504eaea3d9408fc69ae7eb2d14095643c5b": "Curve",
    "0xc8c9cfad493cffffd6684da0f45d141e33b6ef27": "1inch",
    "0x0d67b27c340887da87344c344c20ce60a8eb84c5": "Maverick",
    #"0x9c594c2e2e2e5aa300be12596215188c324c3e7c": "Spectra",
    "0x99a58482bd75cbab83b27ec03ca68ff489b5788f": "Curve",
    "0xa8376f53391a041c8236a232f7f019ea76eed86d": "Odos",
    "0x1a22c1886196101babcac1bf4223294ab04c1d66": "Odos",
    #"0x329c54289ff5d6b7b7dae13592c6b1eda1543ed4": "Marc Zeller",
    "0xc48c6d400de27081df85d55100c7b06f190c764a": "Maverick",
    "0x635ef0056a597d13863b73825cca297236578595": "Curve",
    "0x0b8a49d816cc709b6eadb09498030ae3416b66dc": "1inch",
    "0x1d7405df25fd2fe80390da3a696dcfd5120ca9ce": "ParaSwap",
    #"0x0f4a1d7fdf4890be35e71f3e0bbc4a0ec377eca3": "",
    "0x8dd418fa80b7f37856e36199e67013f00e5d7a61": "Maverick",
    #"0x6e7058c91f85e0f6db4fc9da2ca41241f5e4263f": "Notional",
    #"0xa9c0cded336699547aac4f9de5a11ada979bc59a": "OpenOcean",
    "0xd08d0006f00040b400180f9500b00c5026ac0900": "ParaSwap",
    #"0x55877bd7f2ee37bde55ca4b271a3631f3a7ef121": "OpenOcean",
    #"0x3d20601ac0ba9cae4564ddf7870825c505b69f1a": "Spectra",
    "0x19ea2e6f21bdfc894abf09fa179d59f6c0e0797b": "1inch",
    #"0x93ebc3ca85f96afd72edb914e833fe18888de179": "Yearn V3 Vault",
    "0x63e988f4b30245f9f4ee898531e0be3fee20b170": "1inch",
    "0x6f42b83ecda76a8313fd8a45ca18a3fdfd37bbc7": "Gearbox"
}
contract_names

{'0x00907f9921424583e7ffbfedf84f92b7b2be4977': 'GhoAToken',
 '0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d': 'UniswapV3Pool',
 '0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d': 'GHO Staking',
 '0x9008d19f58aabd9ed0d60971565aa8510560ab41': 'CowSwap',
 '0xdef171fe48cf0115b1d80b88dc8eab59176fee57': 'ParaSwap',
 '0x383e7acd889bf57b0d79a584009cb570534ab518': 'UniswapV3Pool',
 '0xe37e799d5077682fa0a244d46e5649f71457bd09': '1inch',
 '0x54eebc36527fe2e5624051e3c895810d7b68bcfc': 'UniswapV3Pool',
 '0xba12222222228d8ba445958a75a0704d566bf2c8': 'Balancer',
 '0x74de5d4fcbf63e00296fd95d33236b9794016631': 'AirSwap',
 '0xad3b67bca8935cb510c8d18bd45f0b94f54a968f': '1inch',
 '0x8d8404f8cca4c8834ca3cab1e54887ae47724bee': '1inch',
 '0x5756880b6a1eaba0175227bf02a7e87c1e02b28c': 'GHO CCIP',
 '0x22f9dcf4647084d6c31b2765f6910cd85c178c18': '0x Exchange',
 '0x464c71f6c2f760dda6093dcb91c24c39e5d6e18c': 'Aave Collector V2',
 '0xbbec426df87537ad0bec0e899007be441427d0f8': 'UniswapV3Pool',
 '0xf0d4c12a5768d806021f80a262b

In [14]:
minter_txs.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts()

GhoAToken            1142
UniswapV3Pool        1089
Others                693
1inch                 542
GHO Staking           449
CowSwap               298
ParaSwap              267
Curve                 147
Balancer              122
AirSwap               100
Maverick               85
GHO CCIP               58
0x Exchange            53
Aave Collector V2      52
Odos                   48
Kyber Network          37
Li.Fi                  34
Gearbox                34
Name: to, dtype: int64

### Sent to EOAs

In [15]:
other_addresses = [user for user in minter_txs['to'] if user not in contract_names]
len(other_addresses)

693

In [116]:
import asyncio
from web3 import Web3
import concurrent
from tqdm import tqdm

provider = os.environ.get('WEB3_PROVIDER_URL')
w3 = Web3(Web3.HTTPProvider(provider))

def check_address_type(address):
    code = w3.eth.get_code(Web3.to_checksum_address(address))
    return address, len(code) > 0

def process_batch(addresses):
    return [check_address_type(addr) for addr in addresses]

def parallel_address_check(addresses, batch_size=20):
    eoa_count = 0
    contract_count = 0
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        batches = [addresses[i:i + batch_size] for i in range(0, len(addresses), batch_size)]
        futures = [executor.submit(process_batch, batch) for batch in batches]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            batch_results = future.result()
            for address, is_contract in batch_results:
                if is_contract:
                    contract_count += 1
                else:
                    eoa_count += 1
                results.append((address, 'Contract' if is_contract else 'EOA'))

    return results, eoa_count, contract_count

results, eoa_count, contract_count = parallel_address_check(list(set(other_addresses)))

print(f"EOAs: {eoa_count}")
print(f"Contracts: {contract_count}")

100%|████████████████████████████████████████████████████████████████████████████████| 23/23 [00:13<00:00,  1.75it/s]

EOAs: 300
Contracts: 142


In [117]:
first_order_flows = minter_txs.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts().to_dict()
first_order_flows['Sent to other EOA'] = 300
first_order_flows["Others"] = 142
first_order_flows["Held in wallet"] = len(set(hoarders))
first_order_flows

{'GhoAToken': 1142,
 'UniswapV3Pool': 1089,
 'Others': 142,
 '1inch': 542,
 'GHO Staking': 449,
 'CowSwap': 298,
 'ParaSwap': 267,
 'Curve': 147,
 'Balancer': 122,
 'AirSwap': 100,
 'Maverick': 85,
 'GHO CCIP': 58,
 '0x Exchange': 53,
 'Aave Collector V2': 52,
 'Odos': 48,
 'Kyber Network': 37,
 'Li.Fi': 34,
 'Gearbox': 34,
 'Sent to other EOA': 300,
 'Held in wallet': 141}

## Second Order Flows

We filtered dupe transactions in the first order flows so we might have an issue with the numbers not adding up. In particular we would have more second order flows. It might be fine because the expectation is we filtered mostly arbs / MEV but it might not. To avoid issue it'll be better to calculate percentages and then reapply those to the absolute number we have for first order flows.

In [17]:
minter_full_txs = df[df['from'].isin(minters)]

We're only going to check DEX trades

In [64]:
dexes = ['1inch', 'Odos', 'Curve', 'CowSwap', 'ParaSwap', 'Maverick', 'Balancer', 'UniswapV3Pool', '0x Exchange', 'Kyber Network', 'Li.Fi', 'Others']
minter_full_txs['destination'] =  minter_full_txs['to'].apply(lambda x: contract_names.get(x, "Others"))
dex_txs = minter_full_txs[minter_full_txs['destination'].isin(dexes)]
dex_txs

/tmp/ipykernel_78355/1021818953.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,from,to,value,transferIndex,borrowCap,supplyCap,totalVariableDebt,variableBorrowRate,blockNumber,blockTimestamp,transactionHash,mint,destination
10,0x79603115df2ba00659adc63192325cf104ca529c,0x5459ba81d4b375a7b2e34aa83edb1e18c4dc181e,155934142056500000000,10,0,0,81343918969610398129157,15000000000000000000000000,17699326,1689430739,0x9e71943828922409e045ff5c9b9011925d542de62a7d...,False,Others
18,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,28256317265407911651909,18,0,0,277080953174315786866668,15000000000000000000000000,17699363,1689431195,0xc2c13363f315249fc3481b0c94397edbc15269b87bad...,False,Others
38,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,16812655822481809110230,38,0,0,278461016597257517749073,15000000000000000000000000,17699401,1689431675,0x2f8ebe2e50c7b6ddbbaab901e68936d166aef8a01406...,False,Others
50,0x9e9fbeb970383a43d6fba10af4bc0a96e567f83a,0x8e5c57e69ff1a5a34a0358a260d73981f38e924c,100000000000000000000,50,0,0,281362106879226153725806,15000000000000000000000000,17699435,1689432107,0x8f0f230cabf7e21a954886310b70dd0b7c1da1799ff4...,False,Others
55,0x2e21f5d32841cf8c7da805185a041400bf15f21a,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,12977707562003048338489,55,0,0,281162188722374181974933,15000000000000000000000000,17699485,1689432719,0x65ad012bb84f96890b88a8cbc1d1aa889333bab85ef2...,False,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82022,0x0a9972b72910b8668755d0dc016ecc68d1cbbcca,0x16c6521dff6bab339122a0fe25a9116693265353,5000000000000000000000,82022,125000000,0,111471857694936162785433874,60000000000000000000000000,20682692,1725518867,0x84b13ede19fa90a3e9c92e514b89d13d54542d28ddeb...,False,Curve
82065,0xd6c1f6cd5134568979f3c8cf41ed2332cb6b4a74,0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae,2017588478833170819005,82065,125000000,0,111470318026089831961216830,60000000000000000000000000,20683212,1725525167,0x9427f8f1445d4931c5e76ff3dcce55b9639b3a63a897...,False,Li.Fi
82069,0x4f0a01badaa24f762cee620883f16c4460c06be0,0x28104d4f703ee5b5011cefe106f54efd56f33f95,167000999250672397414184,82069,125000000,0,111470450365358614119627153,60000000000000000000000000,20683263,1725525791,0x6d58e97b207f484b57026614a034aef0e1518069f03c...,False,Odos
82078,0x4f0a01badaa24f762cee620883f16c4460c06be0,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,130421901502327359231,82078,125000000,0,111470554709892853590148427,60000000000000000000000000,20683304,1725526283,0x8e8e60ab09cd4a9ffcebf296395599fb7e08d34c0da2...,False,UniswapV3Pool


### Parse token transfers
We're going to look for ERC20 transfers going TO the dumper address in the same transaction and we'll fetch the symbol for those

In [67]:
from web3.exceptions import ContractLogicError
import aiohttp
import pandas as pd

provider = 'https://lb.drpc.org/ogrpc?network=ethereum&dkey=***REMOVED***'
w3 = Web3(Web3.HTTPProvider(provider))

ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "anonymous": False,
        "inputs": [
            {"indexed": True, "name": "from", "type": "address"},
            {"indexed": True, "name": "to", "type": "address"},
            {"indexed": False, "name": "value", "type": "uint256"}
        ],
        "name": "Transfer",
        "type": "event"
    }
]

LP_EVENT_TOPICS = [
    '0x3067048beee31b25b2f1681f88dac838c8bba36af25bfb2b7cf7473a5847e35f' # IncreaseLiquidity UniV3
    '0x26f55a85081d24974e85c6c00045d0f0453991e95873f52bff0d21af4079a768' # AddLiquidity Curve
]

symbols = {'0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2'.lower(): 'MKR'}

def get_token_symbol(token_address):
    contract = w3.eth.contract(address=token_address, abi=ERC20_ABI)
    try:
        symbol = contract.functions.symbol().call()
        return symbol
    except ContractLogicError:
        return "Unknown"

def process_transaction(tx_hash, target_address, destination):
    try:
        receipt = w3.eth.get_transaction_receipt(tx_hash)
    except Exception as e:
        print(e)
        return {}
    events = []
    
    for log in receipt.logs:
        if len(log['topics']) in [3,4] and log['topics'][0].hex() == Web3.keccak(text="Transfer(address,address,uint256)").hex():
            to_address = '0x' + log['topics'][2].hex()[-40:]
            if to_address.lower() == target_address.lower():
                from_address = '0x' + log['topics'][1].hex()[-40:]
                token_address = log['address']
                
                if token_address.lower() in symbols:
                    symbol = symbols[token_address.lower()]
                else:
                    try:
                        symbol = get_token_symbol(token_address)
                        symbols[token_address.lower()] = symbol
                    except Exception as e:
                        print(f'{e}: {token_address} : {tx_hash}')
                        symbol = "Missing"
                
                events.append({
                    'tx_hash': tx_hash,
                    'token_address': token_address,
                    'symbol': symbol,
                    'from': from_address,
                    'to': to_address,
                    'destination': destination,
                })
    
    return events

def process_batch(batch):
    results = []
    for tx_hash, target_address, destination in batch:
        results.extend(process_transaction(tx_hash, target_address, destination))
    return results

def parallel_transaction_processing(tx_hashes, target_addresses, destinations, batch_size=20):
    all_events = []

    tx_address_pairs = list(zip(tx_hashes, target_addresses, destinations))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        batches = [tx_address_pairs[i:i + batch_size] for i in range(0, len(tx_address_pairs), batch_size)]
        futures = [executor.submit(process_batch, batch) for batch in batches]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            batch_events = future.result()
            all_events.extend(batch_events)

    return pd.DataFrame(all_events)

tx_hashes = dex_txs['transactionHash'].to_list()
target_addresses = dex_txs['from'].to_list()
destinations = dex_txs['destination'].to_list()

result_df = parallel_transaction_processing(tx_hashes, target_addresses, destinations)

  0%|                                                                                        | 0/467 [00:00<?, ?it/s]

Transaction with hash: '0x072ac24481a9a8220932a0f7f916593124203708314b2935265f0953d8f7f37f' not found.
Transaction with hash: '0x20d341381aa48ba2bd5f06bcd1228eeaeaedf752c6e281a7398cee3a51d34256' not found.


  2%|█▊                                                                             | 11/467 [00:12<03:25,  2.22it/s]

Transaction with hash: '0x6d8cbc3660da63544190fd832f930d071c95a3713c220bf100c41c8c02b2722a' not found.
Transaction with hash: '0x23394916e4ec60984cb28ed604f3fc830c646111bd78ba7e0e0b579d94130edc' not found.


  3%|██▌                                                                            | 15/467 [00:14<03:43,  2.02it/s]

...

 66%|███████████████████████████████████████████████████▊                          | 310/467 [03:49<01:01,  2.55it/s]

Transaction with hash: '0x236f71e12b2c0d384fcf2ebe243364e6dc12123bba0c0d5f3d3796355bd6a271' not found.


 67%|███████████████████████████████████████████████████▉                          | 311/467 [03:51<02:01,  1.29it/s]

Transaction with hash: '0x872756bddcb1e0447ad1a00439cf5a3ff79ebca17025e5dab62d6751f0d931c8' not found.


100%|██████████████████████████████████████████████████████████████████████████████| 467/467 [05:13<00:00,  1.49it/s]


In [35]:
pd.set_option('display.max_colwidth', None)

In [106]:
result_df

,tx_hash,token_address,symbol,from,to,destination
0,0x0d73f08d46ffd3f12f197a4e03eab32b4ab69457d11d...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xd14f076044414c255d2e82cceb1cb00fb1bba64c,CowSwap
1,0x850d108cf9568bb0a9855b81b746a4e00027e2e83585...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xd14f076044414c255d2e82cceb1cb00fb1bba64c,CowSwap
2,0x84b396c7fad17e16bb22ede6978340d240a1d61c8f93...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
3,0x2cfac29f66bffd5bac296c8b832e631589e2e185e277...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
4,0xc2c13363f315249fc3481b0c94397edbc15269b87bad...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,0x0000000000000000000000000000000000000000,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,Others
...,...,...,...,...,...,...
13211,0x2c4140ca025ba41e0d5a28bb483f6101c310beefbddd...,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,WBTC,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x97e1e6c70fe9f2209792640336534bfc9f93aa7a,UniswapV3Pool
13212,0xd6f7e7e0fbc67557e766698fe936040ec5771d342df1...,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,WBTC,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x97e1e6c70fe9f2209792640336534bfc9f93aa7a,UniswapV3Pool
13213,0xd3e459dcd796c526f22cea607844229cfc3d9b536b8e...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xe705b1d26b85c9f9f91a3690079d336295f14f08,CowSwap
13214,0x264dbb0f1eda1d684a1b70d26f8f6ec82093d6445172...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x5b93d80da1a359340d1f339fb574bdc56763f995,0x2767edfee9f5ba743511ee4187a596b302b43938,UniswapV3Pool


In [73]:
result_df['symbol'].value_counts()[:20]

GHO                   5393
USDC                  2532
USDT                  1168
DAI                    550
WBTC                   456
AAVE                   351
WETH                   329
variableDebtEthGHO     319
wstETH                 252
LINK                   173
UNI-V3-POS             106
LUSD                    91
aEthWETH                86
GHO/bb-a-USD            77
aEthwstETH              66
crvUSD                  60
rETH                    56
USDe                    49
aEthAAVE                48
GHO/USDT/USDC           47
Name: symbol, dtype: int64

In [41]:
dex_txs[dex_txs['transactionHash'] == '0x9d1efc90e836150b21ada9a2cebff14a1d47d74331a6bd5e67530c2cf112152e']

,from,to,value,transferIndex,borrowCap,supplyCap,totalVariableDebt,variableBorrowRate,blockNumber,blockTimestamp,transactionHash,mint,destination
79156,0x686f8d21520f4ecec7ba577be08354f4d1eb8262,0x9008d19f58aabd9ed0d60971565aa8510560ab41,691779724000000000000,79156,116000000,0,106694969088181240637862378,60000000000000000000000000,20641913,1725027179,0x9d1efc90e836150b21ada9a2cebff14a1d47d74331a6bd5e67530c2cf112152e,False,CowSwap


In [118]:
result_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()

to
destination   symbol                   
0x Exchange   AAVE                    7
              BTRFLY                  1
              DAI                     3
              DMT                     1
              GHO                     2
...                                  ..
UniswapV3Pool swETH                   1
              variableDebtEthGHO      4
              variableDebtEthcrvUSD   1
              wTAO                    2
              wstETH                 51

[472 rows x 1 columns]

In [119]:
grouped = result_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()
grouped = grouped.rename(columns={'to': 'count'})
grouped['percentage'] = grouped.groupby(level=0).apply(lambda x: x / x.sum() * 100)
grouped = grouped.reset_index()
grouped = grouped.set_index(['destination', 'symbol'])
grouped

/tmp/ipykernel_78355/3331406645.py:3: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



count  percentage
destination   symbol                                  
0x Exchange   AAVE                       7   10.606061
              BTRFLY                     1    1.515152
              DAI                        3    4.545455
              DMT                        1    1.515152
              GHO                        2    3.030303
...                                    ...         ...
UniswapV3Pool swETH                      1    0.051387
              variableDebtEthGHO         4    0.205550
              variableDebtEthcrvUSD      1    0.051387
              wTAO                       2    0.102775
              wstETH                    51    2.620761

[472 rows x 2 columns]

In [76]:
tokens_df = result_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()

In [77]:
tokens_df.loc['UniswapV3Pool']

,to
symbol,
AAVE,124
ACX,4
ANKR,1
ARB,2
ARES,1
...,...
swETH,1
variableDebtEthGHO,4
variableDebtEthcrvUSD,1


In [78]:
grouped.loc['Curve']

,count,percentage
symbol,,
AAVE,1,0.442478
CRV,2,0.884956
DAI,5,2.212389
DOLA,1,0.442478
ETHx,1,0.442478
FRAX,3,1.327434
FXS,1,0.442478
GHO,2,0.884956
GHOUSDe,4,1.769912


In [29]:
set(result_df['symbol'][result_df['symbol'].str.lower().str.startswith('var')].to_list())

{'variableDebtEthGHO', 'variableDebtEthUSDC', 'variableDebtEthcrvUSD'}

In [107]:
LP_SYMBOLS = {'UNI-V3-POS','ECLP-GHO-GYD',
 'ECLP-GHO-USDC',
              'GHOBTCwstE',
 'ECLP-GHO-USDC-2',
 'GHO-3POOL-BPT',
 'GHO/bb-a-USD',
              'auraGHO/USDT/USDC-vault',
 'MBP-GHO-USDC-15-R1',
 'MBP-GHO-USDC-18-R1',
 'MBP-GHO-USDC-21',
 'MBP-GHO-USDC-21-R1',
              'SPT-PT/IBT-f',
 'MBP-GHO-USDC-23-R1',
 'MBP-GHO-USDC-4-R1',
 'UNI-V3-POS',
 'USDC-DAI-USDT',
 'bb-a-USD',
 'crvUSDGHO-f',
              'GHO/bb-a-USD',  'GHO/LUSD', 'GHO/USDT/USDC', 'GHO/bb-a-USD', 'GHO/USDT/USDC', 'GHOUSDe', 'fxUSDGHO', 'crvUSDGHO-f', 'GHOcrvUSD', 'GHOUSDe', '80wstETH/20GHO'}

ETH_AND_LSTS = {'boxETH',
 'cbETH',
 'ezETH',
 'osETH',
 'rETH',
 'ETH2X',
 'sfrxETH',
 'stETH',
 'swETH',
 'ETHx',
 'WETH',
 'weETH',
 'wstETH'}

BTC = {'LBTC', 'WBTC', 'tBTC'}

OTHER_STABLES = { 'mkUSD',
 'rgUSD',
 'sUSDe', 
 'fxUSD',
                 'sUSD',
 'stUSD', 
 'bb-a-USD',
 'USDP', 
 'PYUSD',
 'DOLA',
 'crvUSD',
 'USD0',
 'USD3',
 'USDA',
                 'FRAX',
 'USDe',
 'GUSD',
                 'sDAI'
 }

A_TOKENS = {
 'aEthAAVE',
 'aEthDAI',
 'aEthUSDC',
    'aEthETHx',
 'aEthUSDT',
 'aEthUSDe',
 'aEthWBTC',
 'aEthWETH',
 'aEthcrvUSD',
 'aEthrETH',
 'aEthwstETH'}

GHO = {'pGHO', 'YT-sw-stkGHO-1732492824', 'GHO-agg', 'nGHO', 'pdGHO', 'PT-sw-stkGHO-1732492824'}

VAR_DEBT_TOKENS = {'variableDebtEthGHO', 'variableDebtEthUSDC', 'variableDebtEthcrvUSD', 'variableDebtEthDAI'}

MAJORS = {'GHO', 'USDC', 'USDT', 'DAI', 'AAVE', 'LINK', 'LUSD'}


In [105]:
result_df[~result_df['symbol'].isin(VAR_DEBT_TOKENS | A_TOKENS | BTC | OTHER_STABLES | ETH_AND_LSTS | LP_SYMBOLS | MAJORS | GHO )]['symbol'].value_counts()[:50]

MKR                35
CRV                34
stkAAVE            23
UNI                20
LDO                14
RPL                13
CVX                13
EURe               12
AURA               11
DIP                10
SNX                 8
FXS                 8
OHM                 7
BITCOIN             7
TSUKA               6
PEPE                6
MATIC               6
sdCRV               5
ILV                 5
MPL                 5
wTAO                5
ENA                 5
BAL                 5
NOTE                4
MPv2                4
ONDO                4
SOL                 4
ACX                 4
RNDR                4
cvxCRV              4
ENS                 4
PENDLE              4
RLB                 4
R                   4
SHIB                3
WOJAK               3
JPGD                3
GNO                 3
RIO                 3
MAV                 2
INJ                 2
EUL                 2
aAAVE               2
GEAR                2
variableDebtDAI     2
RARI      

### Relabelling our symbols

In [110]:
def relabel(label):
    if label in A_TOKENS:
        return 'aTokens'
    elif label in MAJORS:
        return label
    elif label in VAR_DEBT_TOKENS:
        return 'Variable Debt Tokens'
    elif label in OTHER_STABLES:
        return 'Other Stablecoins'
    elif label in BTC:
        return 'BTC'
    elif label in ETH_AND_LSTS:
        return 'ETH/LSTs/LRTs'
    elif label in GHO:
        return 'GHO-Yield Aggregation'
    elif label in LP_SYMBOLS:
        return 'Providing Liquidity'
    return 'Other Cryptos'

label_df = result_df.copy()
label_df['symbol'] = label_df['symbol'].apply(relabel)

label_df

,tx_hash,token_address,symbol,from,to,destination
0,0x0d73f08d46ffd3f12f197a4e03eab32b4ab69457d11d...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xd14f076044414c255d2e82cceb1cb00fb1bba64c,CowSwap
1,0x850d108cf9568bb0a9855b81b746a4e00027e2e83585...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xd14f076044414c255d2e82cceb1cb00fb1bba64c,CowSwap
2,0x84b396c7fad17e16bb22ede6978340d240a1d61c8f93...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Providing Liquidity,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
3,0x2cfac29f66bffd5bac296c8b832e631589e2e185e277...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Providing Liquidity,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
4,0xc2c13363f315249fc3481b0c94397edbc15269b87bad...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Providing Liquidity,0x0000000000000000000000000000000000000000,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,Others
...,...,...,...,...,...,...
13211,0x2c4140ca025ba41e0d5a28bb483f6101c310beefbddd...,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,BTC,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x97e1e6c70fe9f2209792640336534bfc9f93aa7a,UniswapV3Pool
13212,0xd6f7e7e0fbc67557e766698fe936040ec5771d342df1...,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,BTC,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x97e1e6c70fe9f2209792640336534bfc9f93aa7a,UniswapV3Pool
13213,0xd3e459dcd796c526f22cea607844229cfc3d9b536b8e...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xe705b1d26b85c9f9f91a3690079d336295f14f08,CowSwap
13214,0x264dbb0f1eda1d684a1b70d26f8f6ec82093d6445172...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x5b93d80da1a359340d1f339fb574bdc56763f995,0x2767edfee9f5ba743511ee4187a596b302b43938,UniswapV3Pool


In [111]:
label_df['symbol'].value_counts()

GHO                      5393
USDC                     2532
USDT                     1168
ETH/LSTs/LRTs             685
DAI                       550
BTC                       460
Other Cryptos             449
Providing Liquidity       406
AAVE                      351
Variable Debt Tokens      331
Other Stablecoins         272
aTokens                   268
LINK                      173
LUSD                       91
GHO-Yield Aggregation      87
Name: symbol, dtype: int64

In [120]:
grouped = label_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()
grouped = grouped.rename(columns={'to': 'count'})
grouped['percentage'] = grouped.groupby(level=0).apply(lambda x: x / x.sum() * 100)
grouped = grouped.reset_index()
grouped = grouped.set_index(['destination', 'symbol'])
grouped

/tmp/ipykernel_78355/376489346.py:3: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



count  percentage
destination   symbol                                 
0x Exchange   AAVE                      7   10.606061
              BTC                       6    9.090909
              DAI                       3    4.545455
              ETH/LSTs/LRTs             8   12.121212
              GHO                       2    3.030303
...                                   ...         ...
UniswapV3Pool Providing Liquidity     101    5.190134
              USDC                    710   36.485098
              USDT                    312   16.032888
              Variable Debt Tokens      5    0.256937
              aTokens                   5    0.256937

[136 rows x 2 columns]

## Sankey Chart

First let's reformat with a consistant format for 1st and 2nd order

In [121]:
first_order = [{'from': 'Minting', 'to': k, 'amount': v} for k, v in first_order_flows.items()]

In [122]:
first_order

[{'from': 'Minting', 'to': 'GhoAToken', 'amount': 1142},
 {'from': 'Minting', 'to': 'UniswapV3Pool', 'amount': 1089},
 {'from': 'Minting', 'to': 'Others', 'amount': 142},
 {'from': 'Minting', 'to': '1inch', 'amount': 542},
 {'from': 'Minting', 'to': 'GHO Staking', 'amount': 449},
 {'from': 'Minting', 'to': 'CowSwap', 'amount': 298},
 {'from': 'Minting', 'to': 'ParaSwap', 'amount': 267},
 {'from': 'Minting', 'to': 'Curve', 'amount': 147},
 {'from': 'Minting', 'to': 'Balancer', 'amount': 122},
 {'from': 'Minting', 'to': 'AirSwap', 'amount': 100},
 {'from': 'Minting', 'to': 'Maverick', 'amount': 85},
 {'from': 'Minting', 'to': 'GHO CCIP', 'amount': 58},
 {'from': 'Minting', 'to': '0x Exchange', 'amount': 53},
 {'from': 'Minting', 'to': 'Aave Collector V2', 'amount': 52},
 {'from': 'Minting', 'to': 'Odos', 'amount': 48},
 {'from': 'Minting', 'to': 'Kyber Network', 'amount': 37},
 {'from': 'Minting', 'to': 'Li.Fi', 'amount': 34},
 {'from': 'Minting', 'to': 'Gearbox', 'amount': 34},
 {'from'

In [123]:
second_order = []
for protocol, amount in first_order_flows.items():
    if protocol not in dexes:
        continue
    for token in grouped.loc[protocol].sort_values('percentage', ascending=False).index:
        second_order.append(
            {'from': protocol,
            'to': token,
            'amount': int(grouped.loc[protocol,token].percentage / 100 * amount)}
        )
second_order

[{'from': 'UniswapV3Pool', 'to': 'USDC', 'amount': 397},
 {'from': 'UniswapV3Pool', 'to': 'USDT', 'amount': 174},
 {'from': 'UniswapV3Pool', 'to': 'BTC', 'amount': 90},
 {'from': 'UniswapV3Pool', 'to': 'Other Cryptos', 'amount': 83},
 {'from': 'UniswapV3Pool', 'to': 'ETH/LSTs/LRTs', 'amount': 80},
 {'from': 'UniswapV3Pool', 'to': 'AAVE', 'amount': 69},
 {'from': 'UniswapV3Pool', 'to': 'DAI', 'amount': 63},
 {'from': 'UniswapV3Pool', 'to': 'Providing Liquidity', 'amount': 56},
 {'from': 'UniswapV3Pool', 'to': 'LINK', 'amount': 41},
 {'from': 'UniswapV3Pool', 'to': 'Other Stablecoins', 'amount': 12},
 {'from': 'UniswapV3Pool', 'to': 'GHO', 'amount': 7},
 {'from': 'UniswapV3Pool', 'to': 'LUSD', 'amount': 6},
 {'from': 'UniswapV3Pool', 'to': 'Variable Debt Tokens', 'amount': 2},
 {'from': 'UniswapV3Pool', 'to': 'aTokens', 'amount': 2},
 {'from': 'Others', 'to': 'GHO', 'amount': 85},
 {'from': 'Others', 'to': 'USDC', 'amount': 14},
 {'from': 'Others', 'to': 'Variable Debt Tokens', 'amount':

In [124]:
label_df.drop_duplicates(['from', 'to'])['symbol'].value_counts().to_list()
grouped.loc['UniswapV3Pool'].sort_values('percentage', ascending=False)

,count,percentage
symbol,,
USDC,710,36.485098
USDT,312,16.032888
BTC,161,8.273381
Other Cryptos,149,7.656732
ETH/LSTs/LRTs,143,7.348407
AAVE,124,6.372045
DAI,113,5.806783
Providing Liquidity,101,5.190134
LINK,74,3.802672


In [125]:
import json

with open('first_order.json', 'w') as f:
    json.dump(sorted(first_order, key=lambda x: x['amount'], reverse=True), f, indent=2)

with open('second_order.json', 'w') as f:
    json.dump(second_order, f, indent=2)

In [39]:
with open('first_order.json', 'r') as f:
    first_order = json.load(f)

with open('second_order.json', 'r') as f:
    second_order = json.load(f)

In [47]:
import plotly.graph_objects as go
import pandas as pd

def create_sankey(first_order, second_order):
    nodes = {}
    links = []
    
    def add_node(name):
        if name not in nodes:
            nodes[name] = len(nodes)
        return nodes[name]
    
    for item in first_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    for item in second_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    node_list = sorted(nodes.items(), key=lambda x: x[1])
    node_names = [name for name, _ in node_list]
    
    link_sources, link_targets, link_values = zip(*links)
    
    sankey = go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_names,
            color="blue"
        ),
        link=dict(
            source=link_sources,
            target=link_targets,
            value=link_values
        )
    )
    
    layout = go.Layout(
        title_text="Stablecoin Flow Sankey Diagram",
        font_size=10
    )
    
    fig = go.Figure(data=[sankey], layout=layout)
    return fig

sankey_fig = create_sankey(first_order, second_order)

sankey_fig.write_html("sankey_plot.html")
from IPython.display import IFrame
IFrame(src="sankey_plot.html", width=900, height=600)

In [48]:
import plotly.graph_objects as go

def create_sankey(first_order, second_order):
    nodes = {}
    links = []
    
    def add_node(name):
        if name not in nodes:
            nodes[name] = len(nodes)
        return nodes[name]
    
    for item in first_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    for item in second_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    node_list = sorted(nodes.items(), key=lambda x: x[1])
    node_names = [name for name, _ in node_list]
    
    link_sources, link_targets, link_values = zip(*links)
    
    sankey = go.Sankey(
        arrangement='snap',
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_names,
            color="blue",
            align='left'
        ),
        link=dict(
            source=link_sources,
            target=link_targets,
            value=link_values,
            arrowlen=15
        )
    )
    
    layout = go.Layout(
        title_text="Stablecoin Flow Sankey Diagram",
        font_size=10
    )
    
    fig = go.Figure(data=[sankey], layout=layout)
    return fig

sankey_fig = create_sankey(first_order, second_order)

sankey_fig.write_html("sankey_plot.html")
from IPython.display import IFrame
IFrame(src="sankey_plot.html", width=900, height=600)